# Feature Engineering


## Case: Lending Club

Mais um conjunto de dados do Kaggle. O desafio pode ser visto [aqui](https://www.kaggle.com/wendykan/lending-club-loan-data). Lá, estão disponíveis várias das soluções (Kernels) realizadas para esse desafio. 

O Lending Club (ou Clube de empréstimos, em tradução livre) é uma empresa de empréstimo peer-to-peer com sede nos Estados Unidos, na qual os investidores fornecem fundos para potenciais tomadores de empréstimos e os investidores obtêm um lucro, dependendo do risco que correm (a pontuação de crédito do tomador). O Lending Club faz a "ponte" entre investidores e quem precisa de empréstimo.

<img src="../images/loan.png" width = 600/>


Os dados que vamos trabalhar neste notebook contém dados completos de empréstimos realizados entre 2007-2015.

## Download dos dados

O arquivo de dados original (que está lá no Kaggle), tem 1.2Gb. Fiz um filtro para usarmos somente as linhas referentes a empréstimos que já foram liquidados:

In [1]:
#df_complete = pd.read_csv('loan_complete.csv')
#df = df_complete[(df_complete.loan_status != 'Current')]
#df.to_csv(loan.csv)

Ainda assim, o arquivo tem 800Mb (e 329 se comprimirmos em um zip) e o GitHub só aceita arquivos de até 50M, pois o objetivo central do Git é armazenar o histórico de CÓDIGOS! 

O tamanho dos dados ser superior a esse limite é extremamente comum e por isso, daqui para frente vou manter dados no Google Drive e inserir na aula o download deles.

O arquivo dos dados que trabalharemos neste notebook tem 329Mb e com essa biblioteca do Google Drive `google_drive_downloader`, conseguimos acompanhar o progresso do download com o parâmetro `showsize = True`.

<!--- !pip install googledrivedownloader ---> 

In [2]:
import os
import zipfile
from google_drive_downloader import GoogleDriveDownloader as gdd

def download_dataset(dataset_path, google_id):
    if not os.path.exists(dataset_path):
        gdd.download_file_from_google_drive(file_id=google_id,
                                            dest_path=dataset_path, 
                                            showsize = True)
        ''
def unzip_dataset(dataset_path, unzipped_dataset_path):
    if not os.path.exists(unzipped_dataset_path):
        print("Unzipping ...")
        with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
            zip_ref.extractall('../data/')
        print("Done!")

dataset_path = '../data/loan.zip'
google_id = '1GrXYO6WjPAMwb1czTtgsdwU0_VtYBuKQ'
download_dataset(dataset_path, google_id)

unzipped_dataset_path = '../data/loan.csv'
unzip_dataset(dataset_path, unzipped_dataset_path)

## Antes de começarmos: análise exploratória

A análise exploratória, pelo menos uma básica, temos que fazer SEMPRE que nos depararmos com um novo conjunto de dados. Vamos novamente usar as operações que aprendemos para entender um pouco do contexto desses dados, principalmente as colunas que queremos selecionar e as características que queremos/podemos extrair.

In [2]:
import numpy as np
import pandas as pd

#maximo de colunas
pd.set_option("display.max_rows",150)

In [3]:
df = pd.read_csv('../data/loan.csv', low_memory=False)
df.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,100,NaN,NaN,30000,30000,30000.0,36 months,22.35,1151.16,D,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,152,NaN,NaN,40000,40000,40000.0,60 months,16.14,975.71,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,170,NaN,NaN,20000,20000,20000.0,36 months,7.56,622.68,A,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,186,NaN,NaN,4500,4500,4500.0,36 months,11.31,147.99,B,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,215,NaN,NaN,8425,8425,8425.0,36 months,27.27,345.18,E,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


Vamos dar uma olhada na lista das colunas:

In [7]:
df.memory_usage()

Index                                               80
Unnamed: 0                                    10727784
id                                            10727784
member_id                                     10727784
loan_amnt                                     10727784
funded_amnt                                   10727784
funded_amnt_inv                               10727784
term                                          10727784
int_rate                                      10727784
installment                                   10727784
grade                                         10727784
sub_grade                                     10727784
emp_title                                     10727784
emp_length                                    10727784
home_ownership                                10727784
annual_inc                                    10727784
verification_status                           10727784
issue_d                                       10727784
loan_statu

In [5]:
list(df.columns)

['Unnamed: 0',
 'id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_a

Para acessarmos as explicações de cada coluna, temos o arquivo de metadata:

In [6]:
metadata = pd.read_excel('../data/loan_metadata.xlsx')
pd.set_option("display.max_colwidth",500)
metadata

,LoanStatNew,Description
0,acc_now_delinq,The number of accounts on which the borrower is now delinquent.
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan application
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by the borrower during registration.
5,annual_inc_joint,The combined self-reported annual income provided by the co-borrowers during registration
6,application_type,Indicates whether the loan is an individual application or a joint application with two co-borrowers
7,avg_cur_bal,Average current balance of all accounts
8,bc_open_to_buy,Total open to buy on revolving bankcards.
9,bc_util,Ratio of total current balance to high credit/credit limit for all bankcard accounts.


Temos mais de 1 milhão de tomadores de empréstimos nessa base, ou seja, cada linha (instância) temos as características (features) de uma pessoa que pediu empréstimo:

In [7]:
len(df)

1340973

## Features numéricas
Num primeiro momento, vamos selecionar do nosso conjunto algumas *features* (colunas) numéricas, pois comumente, modelos de *machine learning* só aceitam esse tipo de dado.

In [8]:
initial_vars = ['loan_amnt', 'int_rate', 'installment', 'annual_inc',
       'dti', 'delinq_2yrs', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc']

In [9]:
numeric_df = df[initial_vars]

Se a feature é numérica, posso simplesmente usar? 

- Exemplo:

<img src="../images/exe.png" alt="exemplo" width="500"/>


### Padronização e normalização

![standardization](../images/standardization.jpg)

- Padronização ou regularização: uma adequação bastante usada quando lidamos com features numéricas é mapear os valores de uma distribuição para valores de uma distribuição normal padrão para que, independentemente dos valores que temos na distruição, tenhamos a mesma **grandeza** de valores.
- Distribuição normal padrão: uma distribuição normal que tem média 0 e desvio padrão 1.
- Cálculo: para realizar esse mapeamento, calcula-se a diferença entre cada um dos valores e a média e dividimos essa diferença pelo desvio padrão.

<img src="../images/standardization-calc.jpg" alt="calculo_padronizacao" width="400"/>

- Exemplo: 

![standardization_exe](../images/standardization-exe.jpg)

- **Normalização**: tem como objetivo colocar os valores da distribuição dentro do intervalo de 0 e 1 ou, caso tenha resultado negativo, entre -1 e 1.

<img src="../images/norm-calc.png" alt="calculo_norm" width="200"/>


#### Atenção!!!
Se a distribuição não é normal (Gaussiana) ou se o desvio padrão é muito pequeno, a melhor escolha é **normalizar**.

#### Padronização/normalização no Python e o Scikit-learn

O pacote [Scikit-learn](https://scikit-learn.org/stable/) (`sklearn`) é uma biblioteca Python para Machine Learning.
Vamos usar **bastante** essa biblioteca. Ela tem muita coisa de ML pronta (e bem feita) e uma documentação exemplar.

In [10]:
import sklearn

Voltando aos nossos dados numéricos, podemos usar a padronização do Scikit-learn...

Mas antes, **precisamos substituir os nulos**:

In [11]:
#verificando a porcentagem de nulos por coluna
numeric_df.isnull().mean().sort_values(ascending=False).head(50)

mths_since_last_record    0.829352
mths_since_last_delinq    0.503395
revol_util                0.000667
dti                       0.000264
inq_last_6mths            0.000022
total_acc                 0.000022
pub_rec                   0.000022
open_acc                  0.000022
delinq_2yrs               0.000022
annual_inc                0.000003
revol_bal                 0.000000
installment               0.000000
int_rate                  0.000000
loan_amnt                 0.000000
dtype: float64

Podemos fazer a substituição pela média ou mediana, por exemplo.

\* Uma forma um pouco mais robusta: média condicionada (selecionar grupos de dados com base em alguma característica e substituir pela média dentro do grupo)

In [12]:
#verificando a média de cada coluna
numeric_df.mean()

loan_amnt                 14472.390570
int_rate                     13.318540
installment                 439.440210
annual_inc                76196.186448
dti                          18.298196
delinq_2yrs                   0.318950
inq_last_6mths                0.665108
mths_since_last_delinq       34.269902
mths_since_last_record       70.480331
open_acc                     11.591134
pub_rec                       0.215651
revol_bal                 16241.638265
revol_util                   51.883883
total_acc                    24.970100
dtype: float64

In [13]:
#verificando a mediana de cada coluna
numeric_df.median()

loan_amnt                 12000.00
int_rate                     12.79
installment                 375.88
annual_inc                65000.00
dti                          17.63
delinq_2yrs                   0.00
inq_last_6mths                0.00
mths_since_last_delinq       31.00
mths_since_last_record       72.00
open_acc                     11.00
pub_rec                       0.00
revol_bal                 11122.00
revol_util                   52.30
total_acc                    23.00
dtype: float64

In [14]:
numeric_df = numeric_df.fillna(numeric_df.mean())

Agora sim, podemos aplicar a padronização usando o [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) do Scikit-learn:

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numeric_df)

/Users/macbook/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/macbook/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [16]:
numeric_df.head(3)

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc
0,30000,22.35,1151.16,100000.0,30.46,0.0,0.0,51.000000,84.000000,11.0,1.0,15603,37.0,19.0
1,40000,16.14,975.71,45000.0,50.53,0.0,0.0,34.269902,70.480331,18.0,0.0,34971,64.5,37.0
2,20000,7.56,622.68,100000.0,18.92,0.0,0.0,48.000000,70.480331,9.0,0.0,25416,29.9,19.0


In [17]:
#os scalers do Scikit-learn retornam listas numpy como resposta
#se quisermos acessar esses dados em forma de Dataframe, precisamos fazer a conversão:
numeric_df_scaled = pd.DataFrame(scaled_data, columns = numeric_df.columns)
numeric_df_scaled.head(3)

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc
0,1.777562,1.887069,2.717009,0.337743,1.089786,-0.362486,-0.691487,1.083150e+00,1.221274e+00,-0.107907,1.297125,-0.028491,-0.607033,-0.497123
1,2.922337,0.589527,2.047224,-0.442630,2.888203,-0.362486,-0.691487,1.840096e-15,6.418554e-15,1.169885,-0.356635,0.835551,0.514544,1.001714
2,0.632787,-1.203212,0.699523,0.337743,0.055718,-0.362486,-0.691487,8.889225e-01,6.418554e-15,-0.472990,-0.356635,0.409285,-0.896604,-0.497123


Existem outros padronizadores/normalizadores, como o [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html), com o qual conseguimos fazer a normalização comentada acima.

## Features categóricas

Em vários algoritmos e modelos de IA, só são aceitos valores numéricos. Então precisamos de uma forma de mapear (*encoders*) as categorias para números.

In [18]:
cat_cols = ['term', 'grade', 'emp_length', 'home_ownership', 'application_type', 
            'verification_status', 'pymnt_plan', 'purpose']

### Label Encoder

Cada valor único da feature categórica é mapeado para um ID único. No Scikit-learn: [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

Também é necessário preencher os nulos para que esse *encoder* funcione, mas podemos preencher com uma string vazia para que todas as linhas para as quais essa informação não existe receba um mesmo ID.

In [19]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

new_df = pd.DataFrame()
for col in cat_cols:
    new_df[col] = df[col].fillna('')
    new_df[col] = le.fit_transform(new_df[col])

select_df = pd.concat([numeric_df, new_df], axis = 1)

In [20]:
select_df[cat_cols].head()

,term,grade,emp_length,home_ownership,application_type,verification_status,pymnt_plan,purpose
0,0,3,6,1,1,1,0,2
1,1,2,11,1,1,2,0,1
2,0,0,2,1,1,0,0,1
3,0,1,2,5,0,0,0,1
4,0,4,4,1,1,2,0,1


**Atenção:** 
- Os IDs são numéricos e, portanto, **implicam uma ordem**! 
- Por isso, só pode ser usado com algoritmos não lineares baseados em árvore.

### One hot encoding

- Usado com a maioria dos algoritmos lineares
- Podemos excluir a primeira coluna evita colinearidade
- Não introduz ordem ao atributo
- Aumenta a **dimensionalidade** do dataset

In [21]:
select_df[cat_cols].max()

term                    1
grade                   6
emp_length             11
home_ownership          5
application_type        1
verification_status     2
pymnt_plan              1
purpose                13
dtype: int64

Vamos usar o [get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) do Scikit-learn.

In [22]:
new_df = pd.DataFrame()
for col in cat_cols: 
    col_dummies = pd.get_dummies(df[col], drop_first=True, prefix = col)
    new_df = pd.concat([new_df, col_dummies], axis = 1)

select_df = pd.concat([numeric_df, new_df], axis = 1)

In [23]:
select_df.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,...,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,30000,22.35,1151.16,100000.0,30.46,0.0,0.0,51.000000,84.000000,11.0,...,0,0,0,0,0,0,0,0,0,0
1,40000,16.14,975.71,45000.0,50.53,0.0,0.0,34.269902,70.480331,18.0,...,0,0,0,0,0,0,0,0,0,0
2,20000,7.56,622.68,100000.0,18.92,0.0,0.0,48.000000,70.480331,9.0,...,0,0,0,0,0,0,0,0,0,0
3,4500,11.31,147.99,38500.0,4.64,0.0,0.0,25.000000,70.480331,12.0,...,0,0,0,0,0,0,0,0,0,0
4,8425,27.27,345.18,450000.0,12.37,0.0,0.0,34.269902,70.480331,21.0,...,0,0,0,0,0,0,0,0,0,0


### Outros enconders - MATERIAL EXTRA!!!

#### Effect Coding
- Bem similar ao *dummy*;
- Substitui por -1 todos os 0s que representam uma das categorias (primeira ou última) no dummy encoding;
- Exemplo nesse [link](https://towardsdatascience.com/understanding-feature-engineering-part-2-categorical-data-f54324193e63).

- [Outro exemplo](https://stats.idre.ucla.edu/other/mult-pkg/faq/general/faqwhat-is-effect-coding/).

#### Bin-counting 
- Maldição da dimensionalidade, categorias "grandes" (com muitos valores distintos);
- Usa probabilidade baseada em informação estatística da relação do valor com o target;
- Exemplo simplificado do [link 2](https://towardsdatascience.com/understanding-feature-engineering-part-2-categorical-data-f54324193e63): construir valores de probabilidade de um ataque DDoS ser causado por qualquer um dos endereços IP baseado em **dados históricos** de endereços IP relacionados a ataques DDoS.

#### Feature Hashing Scheme
- Maldição da dimensionalidade;
- **Função Hash** tipicamente usada com um número pré-definido de features resultantes (vetor de comprimento pré-definido) de forma que os valores de hash dos valores da categoria sejam usados como índices nesse vetor predefinido e os valores sejam atualizados de acordo.
- Funções Hash mapeiam uma grande quantidade de valores em uma quantidade finita (e pequena) de valores -> colisões.
- [Explicação básica sobre funções hash](https://www.techtudo.com.br/artigos/noticia/2012/07/o-que-e-hash.html)
- [Função Scikit Learn](https://scikit-learn.org/0.19/modules/generated/sklearn.feature_extraction.FeatureHasher.html)

## Outras práticas para CRIAÇÃO de features

- A partir de qualquer tipo de feature (numérica ou categórica), podemos criar outras features, tanto derivando informações (ex: a partir da taxa de juros anual derivar a taxa mensal) quanto enriquecendo o conjunto de dados (ex: cruzando as informações que eu tenho no meu conjunto de dados para trazer novas informações de outro conjunto).

- Um exemplo é quando há valores muito raros e próximos de valores mais numerosos, pois podemos agrupar esses valores por ter maior interesse em um grupo - ex: público alvo por idade.

- Outro exemplo bastante comum é usar **dados geográficos** para criar features. Um exemplo no nosso case: mapear a coluna do estado (`add_state`) para uma coluna de região (que dei o nome de `region`):

In [ ]:
df['addr_state'].unique()

west = ['CA', 'OR', 'UT','WA', 'CO', 'NV', 'AK', 'MT', 'HI', 'WY', 'ID']
south_west = ['AZ', 'TX', 'NM', 'OK']
south_east = ['GA', 'NC', 'VA', 'FL', 'KY', 'SC', 'LA', 'AL', 'WV', 'DC', 'AR', 'DE', 'MS', 'TN' ]
mid_west = ['IL', 'MO', 'MN', 'OH', 'WI', 'KS', 'MI', 'SD', 'IA', 'NE', 'IN', 'ND']
north_east = ['CT', 'NY', 'PA', 'NJ', 'RI','MA', 'MD', 'VT', 'NH', 'ME']

df['region'] = np.nan

def finding_regions(state):
    ...

- Enriquecimento de dados geográficos: podemos agregar mais informação a partir dos dados geográficos que temos no conjunto de dados. [Nesse link](https://medium.com/creditas-tech/incrementando-dados-geogr%C3%A1ficos-com-o-censo-nacional-do-ibge-54d342c4bdcf) é exemplificado essa técnica com dados do IBGE.

## Material para aprofundamento - MATERIAL EXTRA!!!
Para mais detalhes sobre outros métodos de Feature Engineering veja [esses slides](https://www.dropbox.com/s/mebc9i6kxgm516c/feature-engineering-mlmeetup.pdf?dl=0) do HJ van Veen